In [1]:
# module 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 뼈대

In [2]:
train_acc_id = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\train\\train_activity_704.csv').acc_id
test_acc_id = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\test\\test_activity_704.csv').acc_id

In [3]:
#뼈대 만들기
train_acc_id = train_acc_id.to_frame()
test_acc_id = test_acc_id.to_frame()

In [4]:
train_acc_id.index = train_acc_id['acc_id']
test_acc_id.index = test_acc_id['acc_id']

In [5]:
train_acc_id = train_acc_id.drop(['acc_id'], axis = 1)
test_acc_id = test_acc_id.drop(['acc_id'], axis = 1)

# 트레이드 데이터

In [6]:
def select_2min_value(train_column, test_column):
    train = list(set(train_column))
    test = list(set(test_column))
    
    train.sort()
    test.sort()
    
    result = []
    result.extend(train[0:2])
    result.extend(test[0:2])
    
    result = list(set(result))
    result.sort()
    return result[0:2]

In [7]:
train_trade_load = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\train\\train_trade.csv')
test_trade_load = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\test\\test_trade.csv')

In [8]:
train_trade = train_trade_load
test_trade = test_trade_load

In [9]:
min_two = select_2min_value(train_trade_load.item_amount, test_trade_load.item_amount)

In [10]:
train_trade.item_amount = np.around((train_trade.item_amount - min_two[0])/(min_two[1] - min_two[0]) + 1)
test_trade.item_amount = np.around((test_trade.item_amount - min_two[0])/(min_two[1] - min_two[0]) + 1)

In [11]:
train_goomae = train_trade_load.sort_values(by='target_acc_id', ascending=True)
test_goomae = test_trade_load.sort_values(by='target_acc_id', ascending=True)

train_panmae = train_trade_load.sort_values(by='source_acc_id', ascending=True)
test_panmae = test_trade_load.sort_values(by='source_acc_id', ascending=True)

## 상수
---

In [12]:
items = ['accessory', 'costume', 'gem', 'grocery', 'money', 'weapon']

### 1) 구매 건수, 6가지 아이템 별 구매 건수

In [13]:
#구매컬럼명작성
items_goomae = ['cnt_goomae_' + v for v in items]
items_goomae.insert(0, 'acc_id')

In [14]:
train_cnt_goomae = pd.crosstab(index=train_goomae.target_acc_id, columns=train_goomae.item_type).reset_index()
test_cnt_goomae = pd.crosstab(index=test_goomae.target_acc_id, columns=test_goomae.item_type).reset_index()

In [15]:
#컬럼설정
train_cnt_goomae.columns = items_goomae
test_cnt_goomae.columns = items_goomae

In [16]:
#구매건수추가
train_cnt_goomae['cnt_goomae_total'] = np.sum(train_cnt_goomae.loc[:,'cnt_goomae_accessory':], axis=1)
test_cnt_goomae['cnt_goomae_total'] = np.sum(test_cnt_goomae.loc[:,'cnt_goomae_accessory':], axis=1)

In [17]:
#합치기
train_acc_id = pd.merge(train_acc_id, train_cnt_goomae, how='left', on='acc_id')
test_acc_id = pd.merge(test_acc_id, test_cnt_goomae, how='left', on='acc_id')

### 2) 구매 량, 6가지 아이템 별 구매 량

In [18]:
#구매컬럼명작성
items_goomae = ['amt_goomae_' + v for v in items]
items_goomae.insert(0, 'acc_id')

In [19]:
train_amt_goomae = np.around(pd.crosstab(index=train_goomae.target_acc_id, columns=train_goomae.item_type, aggfunc=sum, values=train_goomae.item_amount).fillna(0)).reset_index()
test_amt_goomae = np.around(pd.crosstab(index=test_goomae.target_acc_id, columns=test_goomae.item_type, aggfunc=sum, values=test_goomae.item_amount).fillna(0)).reset_index()

In [20]:
#컬럼설정
train_amt_goomae.columns = items_goomae
test_amt_goomae.columns = items_goomae

In [21]:
#구매량추가
train_amt_goomae['amt_goomae_total'] = np.sum(train_amt_goomae.loc[:,'amt_goomae_accessory':], axis=1)
test_amt_goomae['amt_goomae_total'] = np.sum(test_amt_goomae.loc[:,'amt_goomae_accessory':], axis=1)

In [22]:
#합치기
train_acc_id = pd.merge(train_acc_id, train_amt_goomae, how='left', on='acc_id')
test_acc_id = pd.merge(test_acc_id, test_amt_goomae, how='left', on='acc_id')

### 1) 판매 건수, 6가지 아이템 별 판매 건수

In [23]:
#판매컬럼명작성
items_panmae = ['cnt_panmae_' + v for v in items]
items_panmae.insert(0, 'acc_id')

In [24]:
train_cnt_panmae = pd.crosstab(index=train_panmae.source_acc_id, columns=train_panmae.item_type).reset_index()
test_cnt_panmae = pd.crosstab(index=test_panmae.source_acc_id, columns=test_panmae.item_type).reset_index()

In [25]:
#컬럼설정
train_cnt_panmae.columns = items_panmae
test_cnt_panmae.columns = items_panmae

In [26]:
#판매건수추가
train_cnt_panmae['cnt_panmae_total'] = np.sum(train_cnt_panmae.loc[:,'cnt_panmae_accessory':], axis=1)
test_cnt_panmae['cnt_panmae_total'] = np.sum(test_cnt_panmae.loc[:,'cnt_panmae_accessory':], axis=1)

In [27]:
#합치기
train_acc_id = pd.merge(train_acc_id, train_cnt_panmae, how='left', on='acc_id')
test_acc_id = pd.merge(test_acc_id, test_cnt_panmae, how='left', on='acc_id')

### 2) 판매 량, 6가지 아이템 별 판매 량

In [28]:
#판매컬럼명작성
items_panmae = ['amt_panmae_' + v for v in items]
items_panmae.insert(0, 'acc_id')

In [29]:
train_amt_panmae = np.around(pd.crosstab(index=train_panmae.source_acc_id, columns=train_panmae.item_type, aggfunc=sum, values=train_panmae.item_amount).fillna(0)).reset_index()
test_amt_panmae = np.around(pd.crosstab(index=test_panmae.source_acc_id, columns=test_panmae.item_type, aggfunc=sum, values=test_panmae.item_amount).fillna(0)).reset_index()

In [30]:
#컬럼설정
train_amt_panmae.columns = items_panmae
test_amt_panmae.columns = items_panmae

In [31]:
#판매량추가
train_amt_panmae['amt_panmae_total'] = np.sum(train_amt_panmae.loc[:,'amt_panmae_accessory':], axis=1)
test_amt_panmae['amt_panmae_total'] = np.sum(test_amt_panmae.loc[:,'amt_panmae_accessory':], axis=1)

In [32]:
#합치기
train_acc_id = pd.merge(train_acc_id, train_amt_panmae, how='left', on='acc_id')
test_acc_id = pd.merge(test_acc_id, test_amt_panmae, how='left', on='acc_id')

### 3) 거래 건수, 거래량

In [35]:
train_acc_id = train_acc_id.fillna(0)
test_acc_id = test_acc_id.fillna(0)

In [37]:
#거래건수
train_acc_id['cnt_trade'] = train_acc_id.cnt_goomae_total + train_acc_id.cnt_panmae_total
test_acc_id['cnt_trade'] = test_acc_id.cnt_goomae_total + test_acc_id.cnt_panmae_total

In [38]:
#거래량
train_acc_id['amt_trade'] = train_acc_id.amt_goomae_total + train_acc_id.amt_panmae_total
test_acc_id['amt_trade'] = test_acc_id.amt_goomae_total + test_acc_id.amt_panmae_total

In [48]:
train_acc_id['is_trade'] = train_acc_id['cnt_trade'] > 0
test_acc_id['is_trade'] = test_acc_id['cnt_trade'] > 0

In [52]:
train_acc_id.to_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\train\\train_trade_new.csv')
test_acc_id.to_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\test\\test_trade_new.csv')